In [1]:
import numpy as np
import cdsapi
from pathlib import Path
import pandas as pd

In [ ]:
path_thermal_data = Path("../data/input_data/Thermal_characteristics_beforeEE.csv")
residential_data = pd.read_csv(path_thermal_data, index_col=0)
residential_data

In [3]:
## Import the required library
# from geopy.geocoders import Nominatim
# coordinates_LA = {}

# for LA in residential_data["Local Authority"].unique():
#     # Initialize Nominatim API
#     geolocator = Nominatim(user_agent="MyApp")
#     location = geolocator.geocode(LA)
#     coordinates_LA[LA] = (location.latitude, location.longitude)
#     # print(f"{LA} -- the latitude is: {location.latitude} and the longitude is: {location.longitude}.")

# coordinates_LA_df = pd.DataFrame(coordinates_LA).T
# coordinates_LA_df.columns = ["Latitude", "Longitude"]
# coordinates_LA_df.to_csv(r"../data/intermediary/coordinates_LA.csv")
# coordinates_LA_df.head()

In [ ]:
## Load coordinates data

coordinates_LA_df = pd.read_csv(r"../data/intermediary/coordinates_LA.csv", index_col = 0)
coordinates_LA_df

In [ ]:
# http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 was used to get the coordinates of the location, the input coordinate data is a box [N, W, E,S] 
# where N the latitude of the top of the box, W the longitude of the left of the box, E the longitude of the left of the box and S the bottom of the box.
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form was used to generate the code
# select netcdf format
c = cdsapi.Client()

list_las= list(coordinates_LA_df.index)
for LA in list_las:
    box_coordinates = coordinates_LA_df.loc[LA].to_list() + [x-0.01 if x>0 else x+0.01 for x in coordinates_LA_df.loc[LA].to_list()]
    box_coordinates = [round(x, 2) for x in box_coordinates]
    print(LA, box_coordinates)
    path_project:str = str(Path().absolute()).split("code")[0]

    path_netcdf_files = Path(path_project)/"data"/f"{LA}_netcdf_data"
    path_netcdf_files.mkdir(parents=True, exist_ok=True)


    list_years = [2022]# np.arange(2010, 2022)

    list_variables = ['surface_net_solar_radiation', '2m_temperature']#,  '10m_u_component_of_wind', '10m_v_component_of_wind' #


    for year in list_years:
        for variable in list_variables:
            filename = f'{LA}_{variable}_{year}.nc'
            temp_full_path = path_netcdf_files/filename
            temp_result = c.retrieve(
                'reanalysis-era5-land',
                {
                    'format': 'netcdf',
                    'variable': f'{variable}',
                    'year': f'{year}',
                    'month': [
                        '01','02', '03',
                        '04', '05', '06',
                        '07', '08', '09',
                        '10', '11', '12',
                    ],
                    'day': [
                        '01', '02', '03',
                        '04', '05', '06',
                        '07', '08', '09',
                        '10', '11', '12',
                        '13', '14', '15',
                        '16', '17', '18',
                        '19', '20', '21',
                        '22', '23', '24',
                        '25', '26', '27',
                        '28', '29', '30',
                        '31',
                    ],
                    'time': [
                        '00:00', '01:00', '02:00',
                        '03:00', '04:00', '05:00',
                        '06:00', '07:00', '08:00',
                        '09:00', '10:00', '11:00',
                        '12:00', '13:00', '14:00',
                        '15:00', '16:00', '17:00',
                        '18:00', '19:00', '20:00',
                        '21:00', '22:00', '23:00',
                    ],
                    'area': box_coordinates,
                },
                f'{temp_full_path}')